### **Purpose of this project is to create a HistoryQuiz class, that will take a topic as an input, then generates a question and compare the user´s answer with the correct answer generated by AI.**

In [11]:
%pip install langchain
%pip install --upgrade --quiet  text-generation transformers google-search-results numexpr langchainhub sentencepiece jinja2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 10.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
botocore 1.29.165 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.2.0 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[not

In [2]:
from huggingface_hub import login
login()

In [1]:
import langchain
from langchain.prompts import (
    ChatPromptTemplate, 
    PromptTemplate, 
    SystemMessagePromptTemplate, 
    AIMessagePromptTemplate, 
    HumanMessagePromptTemplate,
    FewShotChatMessagePromptTemplate
)
from langchain_community.llms import HuggingFaceHub
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.output_parsers import DatetimeOutputParser



from datetime import datetime

ModuleNotFoundError: ModuleNotFoundError: No module named 'langchain'

In [13]:
with open('api_key.txt', 'r') as file:
    api_key = file.read()

In [14]:
model_repo = "HuggingFaceH4/zephyr-7b-beta"

llm = HuggingFaceHub(
    repo_id=model_repo,
    task="text-generation",
    huggingfacehub_api_token=api_key,
    model_kwargs={
        "max_new_tokens": 100,
        "top_k": 25,
        "temperature": 0.2,
        "repetition_penalty": 1.2,
    },
)

model = ChatHuggingFace(llm=llm);

WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_api_token is not default parameter.
                    huggingfacehub_api_token was transferred to model_kwargs.
                    Please confirm that huggingfacehub_api_token is what you intended.


In [15]:
class HistoryQuiz():

    def create_history_question(self,topic):
        input_text_template = 'Give me a question about World War 2, where correct answer is a year date.'
        output_text_template = 'In what year did the allies landed on Normandy beach?'
        system_text_template = "You are giving quiz questions, where only the year date is a correct answer."
        human_text_template = "Give me a quiz question about {topic}, where correct answer is a year date. Do not tell the correct answer."
        input_template = HumanMessagePromptTemplate.from_template(input_text_template)
        output_template = AIMessagePromptTemplate.from_template(output_text_template)
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_text_template)
        human_message_prompt = HumanMessagePromptTemplate.from_template(human_text_template)
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt,
                                                        input_template,
                                                        output_template, 
                                                        human_message_prompt])
        prompt = chat_prompt.format_prompt(topic=topic).to_messages()
        question = model(prompt).content
        question = question.splitlines()[-1]
        return question

    
    def get_AI_answer(self,question):
        parser = DatetimeOutputParser()
        input_text_template = 'In which year did Michael Phelps win eight gold medals in swimming at the Olympics?'
        system_text_template ='You will answer quiz questions with just a date.\n{format_instructions}'
        output_text_template = '2008-08-08T00:00:00.000000Z'
        template_text = 'Answer the question:\n{question} in datetime format.'
        input_template = HumanMessagePromptTemplate.from_template(input_text_template)
        output_template = AIMessagePromptTemplate.from_template(output_text_template)
        human_message_prompt = HumanMessagePromptTemplate.from_template(template_text)
        system_message_prompt = SystemMessagePromptTemplate.from_template(system_text_template)
        chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, 
                                                        input_template, 
                                                        output_template, 
                                                        human_message_prompt])
        prompt=chat_prompt.format_prompt(question=question, 
                                         format_instructions=parser.get_format_instructions()).to_messages()
        
        answer = model(prompt).content
        answer = answer.splitlines()[-1]
        correct_datetime = parser.parse(answer)
        return correct_datetime

    
    def get_user_answer(self,question):
        print(question)
        print('\n')
        year = int(input("Enter the year"))
        month= int(input('Enter the month (1-12]'))
        day=int(input('Enter the day'))
        user_datetime = datetime(year, month,day)

        return user_datetime
            
    
    def check_user_answer(self, user_answer, ai_answer):
        diff = user_answer - ai_answer
        print(f'The difference between the dates is: {diff}')

quiz = HistoryQuiz()

In [16]:
question = quiz.create_history_question(topic = 'Google')
question

/opt/python/envs/default/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'When was Google founded by Larry Page and Sergey Brin while they were students at Stanford University?'

In [17]:
ai_answer = quiz.get_AI_answer(question)
print(ai_answer)
print(type(ai_answer))

1998-09-07 00:00:00
<class 'datetime.datetime'>


In [18]:
user_answer = quiz.get_user_answer(question)
user_answer

When was Google founded by Larry Page and Sergey Brin while they were students at Stanford University?


Enter the year 2000
Enter the month (1-12] 2
Enter the day 23


datetime.datetime(2000, 2, 23, 0, 0)

In [19]:
quiz.check_user_answer(user_answer=user_answer, ai_answer=ai_answer)

The difference between the dates is: 534 days, 0:00:00
